In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/100-bird-species/birds'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

import os
import warnings
warnings.simplefilter('ignore')

import cv2
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,Conv2D,Lambda,Flatten,Dropout
from tensorflow.keras.layers import Input,MaxPooling2D,BatchNormalization

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from keras.models import load_model
from keras.preprocessing.image import load_img,img_to_array

import tensorflow as tf
from keras.models import Model,Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense,Flatten,BatchNormalization,Activation,Input

from keras.applications import resnet50
from keras.applications.resnet50 import preprocess_input
from keras import optimizers
from keras import regularizers
from keras.callbacks import EarlyStopping,LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau

from keras.preprocessing.image import (ImageDataGenerator, 
                                       img_to_array, 
                                       array_to_img, 
                                       load_img)

In [ ]:
train = '../input/100-bird-species/birds/train'
test = '../input/100-bird-species/birds/test'
val = '../input/100-bird-species/birds/valid'
df = pd.read_csv("../input/100-bird-species/birds/birds.csv")

In [ ]:
df.head()

In [ ]:
df = df.sample(frac=1)
df_for_img = df.head()
df_for_img

In [ ]:
paths = []
for i in range(5):
    x = df_for_img.iloc[i][1].split('\\')
    y = '/'.join(x)
    paths.append(y)
df_path = pd.DataFrame(paths)
df_path.columns=['paths']
df_path

In [ ]:
plt.figure(figsize=(16,11))
folder = '../input/100-bird-species/285 birds/'    
for i in range(5):
    plt.subplot(1,5,i+1)
    x = folder + df_path.iloc[i][0]
    image = plt.imread(x)
    plt.imshow(image)
    plt.title(df_for_img.iloc[i][2])

In [ ]:
size  = 224
batch_size = 256
train_datagen = ImageDataGenerator(
                             #rotation_range=10,
                             #zoom_range=0.1,
                             #horizontal_flip=True,
                             #fill_mode='nearest',
                             #width_shift_range=0.1,
                             #height_shift_range=0.1,
                             #preprocessing_function = preprocess_input,
                             rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        train,  
        target_size=(size, size), 
        batch_size=batch_size,
        class_mode='sparse')

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1/255)
    #preprocessing_function = preprocess_input)

validation_generator = validation_datagen.flow_from_directory(
        val,  
        target_size=(size, size), 
        #color_mode='rgb',
        batch_size=batch_size,
        class_mode='sparse')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1/255)
    #preprocessing_function = preprocess_input)

test_generator = test_datagen.flow_from_directory(
        test,  
        target_size=(size, size),  
        batch_size=batch_size,
        #color_mode='rgb',
        class_mode='sparse',
        shuffle=False)

In [ ]:
from keras.applications import ResNet101V2
convlayer=ResNet101V2(input_shape=(224,224,3),weights='imagenet',include_top=False)
for layer in convlayer.layers:
    layer.trainable=False

In [ ]:
folder = len(os.listdir(train))
folder

In [ ]:
model=Sequential()
model.add(convlayer)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1024,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(275,activation='softmax'))
print(model.summary())

In [ ]:
opt=tf.keras.optimizers.Adam(lr=0.001)
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer=opt)

In [ ]:
history=model.fit(train_generator,validation_data=validation_generator,
         epochs=5)

In [ ]:
#Training
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(['Train','Test'],loc='upper left')
plt.show()
#Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(['Train','Test'],loc='upper left')
plt.show()

# Model Saving

In [ ]:
model.save("./saved_model/")

In [ ]:
model.save("birds_classification_resnet101v2.h5")
loaded_model = load_model('birds_classification_resnet101v2.h5', compile = False)

## Quantization

## Without quantization

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("./saved_model")
tflite_model = converter.convert()

In [ ]:
len(tflite_model)

## with quantization

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("./saved_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

In [ ]:
len(tflite_quant_model)

In [ ]:
with open("tflite_model.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
with open("tflite_quant_model.tflite", "wb") as f:
    f.write(tflite_quant_model)

## Testing

In [ ]:
labels = train_generator.class_indices
labels_reverse = {k:v for v,k in labels.items()}
labels_reverse

In [ ]:
def output(location):
    img = load_img(location, target_size = (224, 224, 3))
    img = img_to_array(img)
    img = img / 255
    img = np.expand_dims(img, [0])
    answer = loaded_model.predict(img)
    y_class = answer.argmax(axis = -1)
    y = " ".join(str(x) for x in y_class)
    y = int(y)
    res = labels_reverse[y]
    return res

In [ ]:
img = '../input/100-bird-species/birds/test/AMERICAN KESTREL/3.jpg'
pic = load_img(img, target_size = (224, 224, 3))
plt.imshow(pic)
output(img)

In [ ]:
img = '../input/100-bird-species/birds/test/BAY-BREASTED WARBLER/2.jpg'
pic = load_img(img, target_size = (224, 224, 3))
plt.imshow(pic)
output(img)

In [ ]:
img = '../input/100-bird-species/birds/valid/AMERICAN COOT/1.jpg'
pic = load_img(img, target_size = (224, 224, 3))
plt.imshow(pic)
output(img)

# Test Accuracy

In [ ]:
model.evaluate(test_generator)